In [ ]:
from pickle import UnpicklingError
from charset_normalizer import from_bytes
import torch
from transformers.modeling_flax_pytorch_utils import load_flax_checkpoint_in_pytorch_model, load_flax_weights_in_pytorch_model
from transformers import T5ForConditionalGeneration, FlaxT5ForConditionalGeneration, AutoTokenizer
from JaxSeq.bucket_manager import open_with_bucket as open
from JaxSeq.models.T5.load import load_params as t5_load_params, ModelLoadMode as T5ModelLoadMode
from JaxSeq.models.T5.interface import T5Inference
from JaxSeq.utils import jsonl_stream, convert_path, load_mesh, get_dtype
from JaxSeq.train import eval_loss
from IPython import embed
import os
import jax
import json
from torch.nn import CrossEntropyLoss
from JaxSeq.data import MaskIterableDataset
from JaxSeq.utils import BlockingStrategy, Padding, Truncation, get_weight_decay_mask, MapIterable, FileOpenIterable
from transformers.generation import GenerationConfig

flax_params, flax_model = t5_load_params(
            model_load_mode=T5ModelLoadMode.PARAMS,
            model_load_path=model_load_path,
            tokenizer="google/flan-t5-xl",
            mesh=load_mesh((1, 1, -1), ('dp', 'fsdp', 'mp')),
            model_dtype=get_dtype(use_fp16=True),
            params_dtype=get_dtype(use_fp16=True)
        )

def load_t5_pytorch_model(params):
    pt_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", architectures=["T5ForConditionalGeneration"])

    start_indices = (0, 0)
    params["lm_head"]["kernel"] = jax.lax.dynamic_slice(params["lm_head"]["kernel"], start_indices, pt_model.state_dict()["lm_head.weight"].T.shape)
    params["shared"]["embedding"] = jax.lax.dynamic_slice(params["shared"]["embedding"], start_indices, pt_model.state_dict()["shared.weight"].shape)

    print("encoder size: ", pt_model.state_dict()["encoder.embed_tokens.weight"].shape)
    print("decoder size: ", pt_model.state_dict()["decoder.embed_tokens.weight"].shape)

    print("shared embedding size: ", params["shared"]["embedding"].shape)
    print("lm_head kernel size: ", params["lm_head"]["kernel"].shape)

    new_model = load_flax_weights_in_pytorch_model(pt_model, params)

    # if changing to shared embedding 
    # new_model.state_dict()["encoder.embed_tokens.weight"] = new_model.state_dict()["shared.weight"]
    # new_model.state_dict()["decoder.embed_tokens.weight"] = new_model.state_dict()["shared.weight"]
    # if changing to language model head
    # new_model.state_dict()["encoder.embed_tokens.weight"] = new_model.state_dict()["lm_head.weight"].T
    # new_model.state_dict()["decoder.embed_tokens.weight"] = new_model.state_dict()["lm_head.weight"].T

    new_model.state_dict()["encoder.embed_tokens.weight"] = torch.zeros_like(new_model.state_dict()["encoder.embed_tokens.weight"])
    new_model.state_dict()["decoder.embed_tokens.weight"] = torch.zeros_like(new_model.state_dict()["decoder.embed_tokens.weight"])
    return new_model

pt_model = load_t5_pytorch_model(flax_params)